In [2]:
# import modules
import numpy as np 
import cv2
import os

In [3]:
# Creating a KNN Model to learn from face dataset.

def distance(X1,X2):        #Euclidean Distance
    return np.sqrt(((X1-X2)**2).sum())            

def KNN(train, test, K=51):
    #print(train[0,:-1])
    dist = []
    for i in range(train.shape[0]):
        ix = train[i,:-1]
        iy = train[i,-1]
        d = distance(test,ix)
        dist.append([d,iy])

    dist = sorted(dist,key = lambda x:x[0])[:K]

    labels = np.array(dist)[:,-1]

    out = np.unique(labels,return_counts = True)
    index = np.argmax([out[1]])

    return out[0][index]


In [4]:
# Load Dataset 

# Path of dataset
dataset_path = './Dataset/'

face_data = []
labels = []
names={}
class_id = 0

for fx in os.listdir(dataset_path):
    if(fx.endswith('.npy')):
        print("Data Loaded Successfully",dataset_path+fx)
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))
        class_id +=1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis = 0)
face_labels = np.concatenate(labels,axis = 0).reshape((-1,1))

trainset = np.concatenate((face_dataset,face_labels),axis = 1)

Data Loaded Successfully ./Dataset/ishu.npy


In [5]:
# Start Recognition Model : Testing
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
eye_cascade = cv2.CascadeClassifier("haarcascade_eye.xml")
font = cv2.FONT_HERSHEY_SIMPLEX

In [7]:
while True:
    # capture image
    satus,frame = cap.read()
    if(satus == False): #if capture failed so pass
        continue
    
    # Convert captured image to gray scale
    g_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #detect face from captured frame/image
    faces = face_cascade.detectMultiScale(g_frame,1.3,5)

    for face in faces:  #loop for all detected faces from current captured image/frame
        x,y,w,h = face

        offset =5
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        out = KNN(trainset,face_section.flatten())     #pass these to our KNN Model

        cv2.putText(frame,names[int(out)],(x,y-10),font,1,(0,255,255),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

        eye_section_gray = g_frame[y:y+h, x:x+w]
        
        #detect eyes of thge person for better understanding.
        eyes = eye_cascade.detectMultiScale(eye_section_gray,1.3,5)

        for eye in eyes:
            ex,ey,ew,eh = eye

            cv2.rectangle(frame,(x+ex,y+ey),(x+ex+ew,y+eh+ey),(0,255,0),2)
            
    cv2.imshow("frame",frame)  #show image

    if(cv2.waitKey(1) & 0xFF == 13):  #if pressed enter key, the loop will break and come outy i.e execution of face recognition stops.
        break

# stop camera
cap.release()

# destroy all created windows
cv2.destroyAllWindows()
